In [1]:
%%time
# Import libraries and set desired options
import numpy as np
import pandas as pd
from scipy.sparse import hstack
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression

Wall time: 41.1 s


In [ ]:
#https://www.kaggle.com/kashnitsky/correct-time-aware-cross-validation-scheme

In [2]:
# A helper function for writing predictions to a file
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [3]:
%%time
times = ['time%s' % i for i in range(1, 11)]
train_df = pd.read_csv('alice\\train_sessions.csv',
                       index_col='session_id', parse_dates=times)
test_df = pd.read_csv('alice\\test_sessions.csv',
                      index_col='session_id', parse_dates=times)
train_df = train_df.sort_values(by='time1')
# Look at the first rows of the training set
train_df.head()

Wall time: 50.5 s


In [4]:
#Transform data into format which can be fed into CountVectorizer
sites = ['site%s' % i for i in range(1, 11)]
train_df[sites].fillna(0).astype('int').to_csv('train_sessions_text.txt', 
                                               sep=' ', 
                       index=None, header=None)
test_df[sites].fillna(0).astype('int').to_csv('test_sessions_text.txt', 
                                              sep=' ', 
                       index=None, header=None)

In [5]:
%%time
#Fit CountVectorizer and trasfrom data with it

cv = CountVectorizer(ngram_range=(1, 3), max_features=60000)
with open('train_sessions_text.txt') as inp_train_file:
    X_train = cv.fit_transform(inp_train_file)
with open('test_sessions_text.txt') as inp_test_file:
    X_test = cv.transform(inp_test_file)
X_train.shape, X_test.shape

Wall time: 50 s


In [6]:
#Save train targets into a separate vector.
y_train = train_df['target'].astype('int').values

In [7]:
#We'll be performing time series cross-validation, see sklearn TimeSeriesSplit and this dicussion on StackOverflow
time_split = TimeSeriesSplit(n_splits=10)

In [8]:
#
[(el[0].shape, el[1].shape) for el in time_split.split(X_train)]

[((23051,), (23051,)),
 ((46102,), (23051,)),
 ((69153,), (23051,)),
 ((92204,), (23051,)),
 ((115255,), (23051,)),
 ((138306,), (23051,)),
 ((161357,), (23051,)),
 ((184408,), (23051,)),
 ((207459,), (23051,)),
 ((230510,), (23051,))]

In [9]:
#Perform time series cross-validation with logistic regression
logit = LogisticRegression(C=1, random_state=17, solver='liblinear')

In [10]:
%%time
cv_scores = cross_val_score(logit, X_train, y_train, cv=time_split, 
                            scoring='roc_auc', n_jobs=-1) # hangs with n_jobs > 1, and locally this runs much faster

Wall time: 2min 8s


In [11]:
cv_scores, cv_scores.mean()

(array([0.82427331, 0.64549526, 0.8820242 , 0.96012223, 0.84373089,
        0.87848007, 0.94454016, 0.85417534, 0.93131298, 0.90788391]),
 0.8672038343890842)

In [12]:
#Train logistic regression with all training data, make predictions for test set and form a submission file
logit.fit(X_train, y_train)


LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=17, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)

In [13]:
logit_test_pred = logit.predict_proba(X_test)[:, 1]
write_to_submission_file(logit_test_pred, 'subm1.csv') # 0.91288


In [18]:
#Now we'll add some time features: indicators of morning, day, evening and night.

def add_time_features(df, X_sparse):
    hour = df['time1'].apply(lambda ts: ts.hour)
    morning = ((hour >= 7) & (hour <= 11)).astype('int')
    day = ((hour >= 12) & (hour <= 18)).astype('int')
    evening = ((hour >= 19) & (hour <= 23)).astype('int')
    night = ((hour >= 0) & (hour <= 6)).astype('int')
    X = hstack([X_sparse, morning.values.reshape(-1, 1), 
                day.values.reshape(-1, 1), evening.values.reshape(-1, 1), 
                night.values.reshape(-1, 1)])
    return X

In [19]:
%%time
X_train_new = add_time_features(train_df.fillna(0), X_train)
X_test_new = add_time_features(test_df.fillna(0), X_test)

Wall time: 7min 48s


In [20]:
X_train_new.shape, X_test_new.shape

((253561, 60004), (82797, 60004))

In [21]:
%%time
#Performing time series cross-validation, we see an improvement in ROC AUC.
cv_scores = cross_val_score(logit, X_train_new, y_train, cv=time_split, 
                            scoring='roc_auc', n_jobs=-1) # hangs with n_jobs > 1, and locally this runs much faster


Wall time: 2min 7s


In [22]:
cv_scores, cv_scores.mean()

(array([0.87283494, 0.7555006 , 0.93147398, 0.97665104, 0.90554704,
        0.93879362, 0.96194229, 0.92844498, 0.95      , 0.94082052]),
 0.9162009010392771)

In [23]:
#Making a new submission, we notice a leaderboard score improvement as well (0.91288 -> 0.93843). Correlated CV and LB improvements is a good justifications for added features being useful and CV scheme being correct.
logit.fit(X_train_new, y_train)



LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=17, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)

In [24]:
logit_test_pred2 = logit.predict_proba(X_test_new)[:, 1]
write_to_submission_file(logit_test_pred2, 'subm2.csv') # 0.93843


In [32]:
#Now we tune regularization parameter C.
c_values = np.logspace(-2, 2, 10)
logit_grid_searcher = GridSearchCV(estimator=logit, param_grid={'C': c_values},
                                  scoring='roc_auc', n_jobs=1, cv=time_split, verbose=1)


In [34]:
%%time
logit_grid_searcher.fit(X_train_new, y_train) # WTF? Locally, it's 3min 30s


Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed: 17.3min finished


Wall time: 17min 28s


GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=10),
       error_score='raise',
       estimator=LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=17, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': array([1.00000e-02, 2.78256e-02, 7.74264e-02, 2.15443e-01, 5.99484e-01,
       1.66810e+00, 4.64159e+00, 1.29155e+01, 3.59381e+01, 1.00000e+02])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=1)

In [36]:
logit_grid_searcher.best_score_, logit_grid_searcher.best_params_


(0.917376467814612, {'C': 0.21544346900318834})

In [37]:
logit_test_pred3 = logit_grid_searcher.predict_proba(X_test_new)[:, 1]
write_to_submission_file(logit_test_pred3, 'subm3.csv') # 0.94242


In [26]:
from sklearn.preprocessing import StandardScaler
popular_alice_sites = [77, 80, 76, 29, 21, 81, 879, 22, 75, 82, 23, 35, 881, 37, 33, 3000, 733, 30, 78, 941, 7832]
intersection_train = 

In [41]:
popular_alice_sites = [77, 80, 76, 29, 21, 81, 879, 22, 75, 82, 23, 35, 881, 37, 33, 3000, 733, 30, 78, 941, 7832]
intersection_train = ((train_df['site1'].apply(lambda s: s in popular_alice_sites).astype('float64') +\
                     train_df['site2'].apply(lambda s: s in popular_alice_sites).astype('float64') +\
                     train_df['site3'].apply(lambda s: s in popular_alice_sites).astype('float64') +\
                     train_df['site4'].apply(lambda s: s in popular_alice_sites).astype('float64') +\
                     train_df['site5'].apply(lambda s: s in popular_alice_sites).astype('float64') +\
                     train_df['site6'].apply(lambda s: s in popular_alice_sites).astype('float64') +\
                     train_df['site7'].apply(lambda s: s in popular_alice_sites).astype('float64') +\
                     train_df['site8'].apply(lambda s: s in popular_alice_sites).astype('float64') +\
                     train_df['site9'].apply(lambda s: s in popular_alice_sites).astype('float64') +\
                     train_df['site10'].apply(lambda s: s in popular_alice_sites).astype('float64'))>9).astype('float64')
intersection_test = ((test_df['site1'].apply(lambda s: s in popular_alice_sites).astype('float64') +\
                     test_df['site2'].apply(lambda s: s in popular_alice_sites).astype('float64') +\
                     test_df['site3'].apply(lambda s: s in popular_alice_sites).astype('float64') +\
                     test_df['site4'].apply(lambda s: s in popular_alice_sites).astype('float64') +\
                     test_df['site5'].apply(lambda s: s in popular_alice_sites).astype('float64') +\
                     test_df['site6'].apply(lambda s: s in popular_alice_sites).astype('float64') +\
                     test_df['site7'].apply(lambda s: s in popular_alice_sites).astype('float64') +\
                     test_df['site8'].apply(lambda s: s in popular_alice_sites).astype('float64') +\
                     test_df['site9'].apply(lambda s: s in popular_alice_sites).astype('float64') +\
                     test_df['site10'].apply(lambda s: s in popular_alice_sites).astype('float64'))>9).astype('float64')
intersection_train = StandardScaler().fit_transform(intersection_train.values.reshape(-1,1))
intersection_test = StandardScaler().fit_transform(intersection_test.values.reshape(-1,1))

In [33]:
%%time
#add start_month feature
start_month_train = train_df['time1'].apply(lambda ts:100 * ts.year + ts.month).astype('float64')
start_month_train = StandardScaler().fit_transform(start_month_train.values.reshape(-1,1))
X_train_mod = hstack([X_train_new, start_month_train])
start_month_test = test_df['time1'].apply(lambda ts:100 * ts.year + ts.month).astype('float64')
start_month_test = StandardScaler().fit_transform(start_month_test.values.reshape(-1,1))
X_test_mod = hstack([X_test_new, start_month_test])

Wall time: 6.31 s


In [37]:
%%time
#full_df['duration'] = full_df[times].max(axis=1)-full_df[times].min(axis=1)
#full_df['duration']=full_df['duration'].apply(lambda ts:ts.seconds).astype('float64')
duration_train = train_df[times].max(axis=1)-train_df[times].min(axis=1)
duration_test = test_df[times].max(axis=1)-test_df[times].min(axis=1)
duration_train = duration_train.apply(lambda ts:ts.seconds).astype('float64')
duration_test = duration_test.apply(lambda ts:ts.seconds).astype('float64')
duration_train

Wall time: 11.7 s


In [38]:
type(duration_train) #= StandardScaler().fit_transform(duration_train.values.reshape(-1,1))
duration_train = StandardScaler().fit_transform(duration_train.values.reshape(-1,1))
duration_test = StandardScaler().fit_transform(duration_test.values.reshape(-1,1))

In [42]:
X_train_mod = hstack([X_train_new, start_month_train, duration_train, intersection_train])
X_test_mod = hstack([X_test_new, start_month_test, duration_test, intersection_test])
X_train_mod.shape, X_test_mod.shape

((253561, 60007), (82797, 60007))

In [44]:
%%time
#Performing time series cross-validation, we see an improvement in ROC AUC.
cv_scores = cross_val_score(logit, X_train_mod, y_train, cv=time_split, 
                            scoring='roc_auc', n_jobs=-1) # hangs with n_jobs > 1, and locally this runs much faster
cv_scores, cv_scores.mean()

Wall time: 2min 52s


In [45]:
cv_scores, cv_scores.mean()

(array([0.87949428, 0.76634631, 0.95092091, 0.97799922, 0.90702422,
        0.94191652, 0.96419839, 0.92952671, 0.95111928, 0.94171334]),
 0.9210259178569732)

In [46]:
%%time
logit.fit(X_train_mod, y_train)

Wall time: 1min 18s


LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=17, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)

In [47]:
logit_test_pred2 = logit.predict_proba(X_test_mod)[:, 1]
write_to_submission_file(logit_test_pred2, 'subm2.csv') # 0.94187

In [48]:
#Now we tune regularization parameter C.
c_values = np.logspace(-2, 2, 10)
logit_grid_searcher = GridSearchCV(estimator=logit, param_grid={'C': c_values},
                                  scoring='roc_auc', n_jobs=-1, cv=time_split, verbose=1)

In [49]:
%%time
logit_grid_searcher.fit(X_train_mod, y_train) # WTF? Locally, it's 3min 30s

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 29.7min finished


Wall time: 30min 29s


GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=10),
       error_score='raise-deprecating',
       estimator=LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=17, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'C': array([1.00000e-02, 2.78256e-02, 7.74264e-02, 2.15443e-01, 5.99484e-01,
       1.66810e+00, 4.64159e+00, 1.29155e+01, 3.59381e+01, 1.00000e+02])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=1)

In [50]:
logit_grid_searcher.best_score_, logit_grid_searcher.best_params_

(0.9230989398081443, {'C': 0.21544346900318834})

In [51]:
logit_test_pred3 = logit_grid_searcher.predict_proba(X_test_mod)[:, 1]
write_to_submission_file(logit_test_pred3, 'subm3.csv') # 0.94589